In [ ]:
#Assignment 7#
#I believe my code is correct, however I got an error throughout dealing with opening the file#
#I used what was provided to open the file#


In [56]:
#7A#
import pandas as pd
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as parq

def get_key(val):
    for key, value in parts.items():
        #if val == str(value):
        if val in value:
            return key
    return "key doesn't exist"

partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )
partitions_val_keys = (
        'A', 'B', 'C-D', 'E-F',
        'G-H', 'I-J', 'K-L', 'M',
        'N', 'O-P', 'Q-R', 'S-T',
        'U', 'V', 'W-X', 'Y-Z'
    )
parts = dict(zip(partitions_val_keys, partitions))
print(parts)

import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil
import pandas as pd
import pygeohash
import s3fs
endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        
    return records
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    
    return flat_record
def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])
df = create_flattened_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

ndf = pd.read_parquet(df, engine='pyarrow')
print(list(ndf.columns.values))

ndf['key'] = ndf['src_airport.iata']+ndf['dst_airport.iata']+ndf['airline.icao']
ndf['partition_value'] = ndf['key'].str[:1]
ndf['kv_key'] = ndf.apply(lambda x: get_key(x.partition_value), axis=1)


table = pa.Table.from_pandas(ndf)


print(table)


{'A': ('A', 'A'), 'B': ('B', 'B'), 'C-D': ('C', 'D'), 'E-F': ('E', 'F'), 'G-H': ('G', 'H'), 'I-J': ('I', 'J'), 'K-L': ('K', 'L'), 'M': ('M', 'M'), 'N': ('N', 'N'), 'O-P': ('O', 'P'), 'Q-R': ('Q', 'R'), 'S-T': ('S', 'T'), 'U': ('U', 'U'), 'V': ('V', 'V'), 'W-X': ('W', 'X'), 'Y-Z': ('Y', 'Z')}


TypeError: cannot construct a FileSource from        airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  Apache   
67661               19016        Apache Air                  Apache   
67662               19016        Apache Air                  Apache   

      airline_iata airline_icao airline_callsign airline_country  \
0               2B          ARD  

Exception ignored in: 'pyarrow._dataset._make_file_source'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/pyarrow/parquet.py", line 1514, in __init__
    fragment = parquet_format.make_fragment(single_file, filesystem)
TypeError: cannot construct a FileSource from        airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  

ArrowInvalid: Called Open() on an uninitialized FileSource

In [57]:
#Assignment 7B#
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

ndf = pd.read_parquet(df, engine='pyarrow')

ndf['key'] = ndf['src_airport.iata']+ndf['dst_airport.iata']+ndf['airline.icao']
ndf['hashed'] = ndf.apply(lambda x: hash_key(x))
ndf['hash_key'] = ndf['hashed'].str[:1]

# For troubleshooting
table = pa.Table.from_pandas(ndf)

ndftable = parq.read_table(partitioned_parquet_file)
print(ndftable)


TypeError: cannot construct a FileSource from        airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  Apache   
67661               19016        Apache Air                  Apache   
67662               19016        Apache Air                  Apache   

      airline_iata airline_icao airline_callsign airline_country  \
0               2B          ARD  

Exception ignored in: 'pyarrow._dataset._make_file_source'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/pyarrow/parquet.py", line 1514, in __init__
    fragment = parquet_format.make_fragment(single_file, filesystem)
TypeError: cannot construct a FileSource from        airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  

ArrowInvalid: Called Open() on an uninitialized FileSource

In [58]:
#Assignment 7C#
import pygeohash
West = pygeohash.encode(45.5945645, -121.1786823)
print(West)
Central = pygeohash.encode(41.1544433, -96.0422378)
print(Central)
East = pygeohash.encode(39.08344, -77.6497145)
print(East)

c21g6s0rs4c7
9z7dnebnj8kb
dqby34cjw922


In [60]:
ndf = pd.read_parquet(df, engine='pyarrow')


ndf['key'] = ndf['src_airport.iata']+ndf['dst_airport.iata']+ndf['airline.icao']
ndf['long'] = ndf['src_airport.longitude']
ndf['lat'] = ndf['src_airport.latitude']
ndf['location'] = ndf.apply(lambda x: get_data_center_val(x.lat, x.long, West, Central, East), axis=1)


ntable = pa.Table.from_pandas(ndf)

print(ntable)

TypeError: cannot construct a FileSource from        airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  Apache   
67661               19016        Apache Air                  Apache   
67662               19016        Apache Air                  Apache   

      airline_iata airline_icao airline_callsign airline_country  \
0               2B          ARD  

Exception ignored in: 'pyarrow._dataset._make_file_source'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/pyarrow/parquet.py", line 1514, in __init__
    fragment = parquet_format.make_fragment(single_file, filesystem)
TypeError: cannot construct a FileSource from        airline_airline_id      airline_name           airline_alias  \
0                     410        Aerocondor  ANA All Nippon Airways   
1                     410        Aerocondor  ANA All Nippon Airways   
2                     410        Aerocondor  ANA All Nippon Airways   
3                     410        Aerocondor  ANA All Nippon Airways   
4                     410        Aerocondor  ANA All Nippon Airways   
...                   ...               ...                     ...   
67658                4178  Regional Express          Qantas Airways   
67659               19016        Apache Air                  Apache   
67660               19016        Apache Air                  

ArrowInvalid: Called Open() on an uninitialized FileSource

In [113]:
#Assignment 7D#
def balance_partitions (keys, num_partitions):
    unique = sorted(set(keys))
    numbervals = len(unique)
    count = (numbervals / num_partitions)+1
    partitions  = []
    parta = 10
    partb = 10
    for i in range(numbervals):
        keyval ={}
        if parta <= count:
            keyval[unique[i]] = partb
            parta = parta + 1
        else:
            parta = 1
            partb = partb + 1
            keyval[unique[i]] = partb
            parta = parta + 1
        partitions .append(keyval)
    return partitions

In [114]:
keys = ['apple', 'grape', 'orange', 'banana']
nummberpartitions = 5
partitions  = balance_partitions(keys, nummberpartitions)
print(partitions )

keys = ['green', 'red', 'orange', 'purple']
nummberpartitions = 4
partitions  = balance_partitions(keys, nummberpartitions)
print(partitions )

[{'apple': 11}, {'banana': 12}, {'grape': 13}, {'orange': 14}]
[{'green': 11}, {'orange': 11}, {'purple': 12}, {'red': 12}]
